In [1]:
import json
import h5py
from load_data import *
from model import *
from plotting import *
import tensorflow as tf 
import os
from numpy import sqrt as sqrt

Importamos los ground truth del conjunto de entrenamiento y test. 

In [2]:
json_train = open('JSON FILES\DTS_SG_part_A.json')
json_test = open('JSON FILES\DTS_SG_part_A_TEST.json')
paths_train = json.load(json_train)
paths_test = json.load(json_test)

#Cargamos la data 
train_GT = mLoad_GT(paths_train)
train_img = mLoad_Img(paths_train)

test_GT = mLoad_GT(paths_test)
test_img = mLoad_Img(paths_test)

In [3]:
f = h5py.File(r'C:\Users\Usuario\Documents\Nueva carpeta\Gaussian-Derivates-Network\training\resume.h5', 'r')

In [4]:
predict = np.asarray(f['predict'])

Construimos la red neuronal. Este sera un prototipo hasta siguiente cambio

In [5]:
#Definimos los sigmas
input_sigma = []
for i in range(6):
    input_sigma.append(0.2015*(sqrt(2))**(i))
    
#Definimos el tamaño de nuestra imagen.   
input_shape = train_img[0,:,:,:].shape

#Creamos el modelo
model = Betsy(input_shape= input_shape, input_sigmas= input_sigma, input_kernel_size=(7, 7))


#model.build_graph(input_shape).summary()
#Compliamos el modelo
model.compile(loss = GAME_loss,
              optimizer = tf.keras.optimizers.Adam(learning_rate = 5e-6), 
              metrics = [sMAE(), RMSE()])

Entrenamos un poco para testeo!

In [ ]:
checkpoint_path = "training/cp-0044.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [ ]:
batch_size = 10

# Create a callback that saves the model's weights
#cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
#                                                 save_weights_only=True,
#                                                 verbose=1)

model.fit(train_img, 
          train_GT, 
          batch_size = batch_size, 
          epochs = 1, 
          validation_data=(test_img, test_GT)) 
          #callbacks=[cp_callback])

In [ ]:
pre_trained = r"training\cp-0050.ckpt"
model.load_weights(pre_trained)

In [ ]:
predict = model(test_img)

In [7]:
IMG = test_img[0,:,:,0]

In [9]:
np.float32(tf.math.reduce_sum(IMG))

334887.75

In [6]:
count_estimate(test_img, test_GT, predict, model)

AttributeError: 'DType' object has no attribute 'kind'

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(predict[0,:,:,:])